In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io as sio
import networkx as nx

In [ ]:
# Generate all 2^n binary strings
def generate_strings(n):
    if n == 1:
        return ["0", "1"]
    ret = []
    all_smaller_strings = generate_strings(n-1)
    ret.extend(["0" + s for s in all_smaller_strings])
    ret.extend(["1" + s for s in all_smaller_strings])
    return ret

# Convert string to vector
def get_vector_from_string(s):
    ret = []
    for digit in s:
        if digit == "0":
            ret.append(-1)
        else:
            ret.append(1)
    return np.array(ret)

# Get ground truth logZ and mode
def ground_truth(A, h, all_strings):
    ret = 0
    n = len(A)
    mx = float("-inf")
    xTAx_plus_hx_list = []
    for s in all_strings:
        x = get_vector_from_string(s)
        xTAx = (x.T @ A @ x) / 2
        hx = h.T @ x
        xTAx_plus_hx = xTAx + hx
        mx = max(mx, xTAx_plus_hx)
        xTAx_plus_hx_list.append(xTAx_plus_hx)
    xTAx_plus_hx_list = [elem - mx for elem in xTAx_plus_hx_list]
    logZ = np.log(np.sum(np.exp(xTAx_plus_hx_list))) + mx
    return logZ, mx

In [ ]:
# Define MRF parameters
cw_range = np.arange(0, 5.25, 0.25)
cw_range[0] = 0.05
cw_len = len(cw_range)
num_samples = 100
n = 20

<h2> Complete graph data generation

In [ ]:
coupling_data = np.zeros((n, n, len(cw_range), num_samples))
bias_data = np.zeros((n, len(cw_range), num_samples))
logZ_data = np.zeros((len(cw_range), num_samples))
modes_data = np.zeros((len(cw_range), num_samples))
all_strings = generate_strings(n)
for i, cw in enumerate(cw_range):
    for sample in range(num_samples):
        print(i, sample)
        A = np.random.uniform(low=-2*cw, high=2*cw, size=(n, n))
        for row in range(n):
            for col in range(n):
                if row == col:
                    A[row, col] = 0
                else:
                    A[row, col] = A[col, row]
        h = np.random.uniform(low=-1, high=1, size=(n))
        logZ, mode = ground_truth(A, h, all_strings)
        coupling_data[:, :, i, sample] = A
        bias_data[:, i, sample] = h
        logZ_data[i, sample] = logZ
        modes_data[i, sample] = mode

<h2> ER graph data generation

In [ ]:
p = 0.5
coupling_data = np.zeros((n, n, cw_len, num_samples))
bias_data = np.zeros((n, cw_len, num_samples))
logZ_data = np.zeros((cw_len, num_samples))
modes_data = np.zeros((cw_len, num_samples))
all_strings = generate_strings(n)
for i, cw in enumerate(cw_range):
    t = 2 * cw / p
    for sample in range(num_samples):
        print(i, sample)
        G = nx.erdos_renyi_graph(n, p)
        A = np.zeros((n, n))
        for edge in G.edges:
            u, v = edge
            value = np.random.uniform(low=-t, high=t)
            A[u, v] = value
            A[v, u] = value
        h = np.random.uniform(low=-1, high=1, size=(n))
        logZ, mode = ground_truth(A, h, all_strings)
        coupling_data[:, :, i, sample] = A
        bias_data[:, i, sample] = h
        logZ_data[i, sample] = logZ
        modes_data[i, sample] = mode